In [1]:
import torch
from torchvision import datasets,models,transforms
from torch import optim
import torch.nn as nn
import numpy as np

In [ ]:
# load dataset
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

data_transforms = {
    "train" : transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.Normalize(mean=mean , std= std),
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(p=0.3)

    ])}